Most up to date top_offender approach, level 1

In [75]:
# %load_ext autoreload
# %reload_ext autoreload
# %autoreload 2

In [76]:
import os
os.chdir('C:\\Users\\WDG1DCR\\Desktop\\my_projects\\data_analysis\\data_cleanse')
# os.chdir('C:\\Users\\gratten\\Documents\\python\\projects\\data_cleanse')

import pandas as pd
from prep import filterx
from prep import sort
# from prep import build
from plot import bar
from utility import generate

In [77]:
fileName = 'filelist_9_02_c'
df_og = pd.read_csv('datasets/' + fileName + '.csv', sep=';')

In [78]:
df = filterx.sld(df_og)

In [79]:
dups = df[df.duplicated(['name_low'], keep=False)]
dups = dups.sort_values(['name_low'])

In [80]:
dups = dups.reset_index()

In [81]:
top_offenders = dups['path'].value_counts().index.to_list()

In [82]:
def build_data(dups, num):
    directory = top_offenders[num]
    
    file_list = []
    for i in range(len(dups['path'])):
        if dups['path'][i] == top_offenders[num]:
            file_list.append(dups['name_low'].iloc[i])
            
    file_paths = {}
    for file in file_list:
        dups_trimmed = dups[(dups['name_low'] == file)]
        path_list = []
        for path in dups_trimmed['path']:
            path_list.append(path)
        file_paths[file] = path_list
        
    totalFiles = 0
    for k, v in file_paths.items():
        totalFiles += len(v)
    
    occur_list = []
    for k, v in file_paths.items():
        if len(v) not in occur_list:
            occur_list.append(len(v))
    
    occur_dict = {}
    for i in occur_list:
        occur_dict[i] = 0
        for v in file_paths.values():
            if i == len(v):
                occur_dict[i] += 1
                
    num_list = []
    occur_list = []
    for i in range(len(occur_dict)):
        key = min(occur_dict.keys())
        num_list.append(key)
        value = occur_dict.pop(key)
        occur_list.append(value)    
        
#     print(totalFiles)
        
    return num_list, occur_list, totalFiles, file_paths

In [83]:
# def batch2(files_to_move):
#     os.chdir('H:/temp/data_cleanse_batch_files')
#     moveFileName = '9_02_files_to_remove_palette_script.txt'
#     moveFile = open(moveFileName, 'w')
#     for i in files_to_move:
#         newLoc_path = i
#         newLoc_path = newLoc_path[7:]
#         print(newLoc_path)
#         newLoc_path = newLoc_path[:newLoc_path.rindex('\\')]
#         newLoc_path = newLoc_path.replace(' ', '')
#         newLoc_path = newLoc_path.replace(',', '')
#         newLoc = 'C:\\Users\\wdg1dcr\\Desktop\\dupBackup\\movedFiles2\\' + newLoc_path
#         moveFile.write(f'md {newLoc} 2> nul & move \"{i}\"*.* {newLoc}\n')
#     moveFile.close()
#     os.chdir('C:\\Users\\WDG1DCR\\Desktop\\my_projects\\data_analysis\\data_cleanse')

In [84]:
################################################
# choose how many top offenders will be analyzed
################################################
count = 500

# num_list, occur_list, totalFiles, file_paths = build_data(dups, num)

In [85]:
len(top_offenders)

557

In [86]:
offender_list = top_offenders#[:count]
# offender_list = top_offenders

In [87]:
# for i in offender_list:
#     print(i)

In [88]:
from collections import defaultdict

# {file1:[<path1 with 'palette'>, <path2 with 'palette'>, etc]}
palette_keep_dict = defaultdict(list)

# {file1:[<path1 without 'palette'>, <path2 without 'palette'>, etc]}
palette_del_dict = defaultdict(list)

for x in range(len(offender_list)):
    num_list, occur_list, totalFiles, file_paths = build_data(dups, x)
#     num_list, occur_list, totalFiles, file_paths = build.data1(dups, x)
    for k, v in file_paths.items():
        for i in v:
            if 'palette' in i.lower():
                palette_keep_dict[k].append(i)
            else:
                palette_del_dict[k].append(i)             

In [89]:
for k, v in palette_keep_dict.items():
    palette_keep_dict[k] = list(set(v))

In [90]:
# for k, v in palette_keep_dict.items():
#     print(k)
#     for i in v:
#         print(i)
#     print()

In [91]:
# FILENAMES THAT OCCUR IN MULTIPLE 'PALETTE' LOCATIONS
count = 0
for k, v in palette_keep_dict.items():
    if len(v) > 1:
        count += len(v)
        print(k)
        for i in v:
            i = i.replace('\\\dcrsolidworks.patec.group\\Engineer1_E', 'R:')
            print(i)
        print()

m6 soc set scr.sldprt
R:\Doc\SolidWorks\SolidWorks Palette\Palette Parts\Hardware\Metric
R:\Doc\SolidWorks\SolidWorks Palette\Palette Parts\Fasteners

nut.sldprt
R:\Doc\SolidWorks\SolidWorks Palette\Palette Parts\Hardware\Inch\metric
R:\Doc\SolidWorks\SolidWorks Palette\Palette Parts\Hardware\Metric
R:\Doc\SolidWorks\SolidWorks Palette\Palette Parts\Hardware

118972 val smc as1201f-m5-06.sldprt
R:\Doc\SolidWorks\SolidWorks Palette\Palette Parts\Valves
R:\Doc\SolidWorks\SolidWorks Palette\Palette Parts\Fittings



In [92]:
# assesss files that occur only one time in a 'palette' directory, deletes all other directories
files_to_move = []
for k, v in palette_del_dict.items():
    if len(palette_keep_dict[k]) == 1:
        for i in v:
            i = i.replace('\\\dcrsolidworks.patec.group\\Engineer1_E', 'R:')
            file_and_path = i + '\\' + k
            files_to_move.append(file_and_path)

In [93]:
# # assesses files that occur in more than one 'palette' directory, removes all but the shortest directory
# files_to_move = []
# for k, v in palette_keep_dict.items():
#     if len(palette_keep_dict[k]) > 1:
#         path_to_keep = min(v, key=len)
#         v.remove(path_to_keep)
#         for path in v:
#             path = path.replace('\\\dcrsolidworks.patec.group\\Engineer1_E', 'R:')
#             file_and_path = path + '\\' + k
#             files_to_move.append(file_and_path)

In [94]:
files_to_move = list(set(files_to_move))

In [95]:
len(files_to_move)

6

In [96]:
###GENERATE BATCH FILE###
output_fname = '9_04_files_to_remove_palette_script'
# generate.batch2(output_fname, file_list)